In [17]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv('sentences_author.csv')

In [18]:
from transformers import BertForSequenceClassification, BertTokenizer
import torch
from torch.nn import Sigmoid

# Load model and tokenizer (ensure they are already initialized)
# checkpoint path: /home/nana/DASP_report_template/model_training/nlp/review_to_theme/results/checkpoint-1810
# bert-base-uncased
model = BertForSequenceClassification.from_pretrained("/home/nana/DASP_report_template/model_training/nlp/review_to_theme/results/final_model", num_labels=11, problem_type="multi_label_classification")
tokenizer = BertTokenizer.from_pretrained("/home/nana/DASP_report_template/model_training/nlp/review_to_theme/results/final_model")

# Make sure the model is in evaluation mode
model.eval()

def predict(input_text, threshold=0.5):
    # Tokenize the input text using tokenizer (handles padding, truncation, etc.)
    inputs = tokenizer(input_text, truncation=True, padding="max_length", max_length=128, return_tensors="pt")

    # Run the model for prediction
    with torch.no_grad():  # Disable gradient calculation during inference
        outputs = model(**inputs)
        logits = outputs.logits

    # Apply sigmoid to logits (since it's multi-label classification)
    sigmoid = Sigmoid()
    probabilities = sigmoid(logits)

    # Apply threshold to get binary predictions (0 or 1)
    predictions = (probabilities > threshold).int()

    # Return the binary predictions (as a list)
    return predictions.squeeze().tolist()

# Example usage
input_text = "Lack of limitations."
predicted_labels = predict(input_text)

print("Predicted Labels:", predicted_labels)


Predicted Labels: [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]


In [19]:
labels = ['ANA', 'BIB', 'DAT', 'EXP', 'INT', 'MET', 'OAL', 'PDI', 'RES', 'RWK', 'TNF']
# Find all indices of 1
indices = [i for i, value in enumerate(predicted_labels) if value == 1]
true_labels = [labels[i] for i in indices]

In [20]:
true_labels

['MET']

In [21]:
def predict_theme_category(text):
    # Load the pretrained model and tokenizer
    model = BertForSequenceClassification.from_pretrained("/home/nana/DASP_report_template/model_training/nlp/review_to_theme/results/final_model", num_labels=11, problem_type="multi_label_classification")
    tokenizer = BertTokenizer.from_pretrained("/home/nana/DASP_report_template/model_training/nlp/review_to_theme/results/final_model")
    model.eval()
    threshold = 0.5
    # Tokenize the input text using tokenizer (handles padding, truncation, etc.)
    inputs = tokenizer(text, truncation=True, padding="max_length", max_length=128, return_tensors="pt")

    # Run the model for prediction
    with torch.no_grad():  # Disable gradient calculation during inference
        outputs = model(**inputs)
        logits = outputs.logits

    # Apply sigmoid to logits (since it's multi-label classification)
    sigmoid = Sigmoid()
    probabilities = sigmoid(logits)

    # Apply threshold to get binary predictions (0 or 1)
    predictions = (probabilities > threshold).int()
    predictions = predictions.squeeze().tolist()

    labels = ['ANA', 'BIB', 'DAT', 'EXP', 'INT', 'MET', 'OAL', 'PDI', 'RES', 'RWK', 'TNF']
    # Find all indices of 1
    indices = [i for i, value in enumerate(predictions) if value == 1]
    true_labels = [labels[i] for i in indices]

    # Return the binary predictions (as a list)
    return true_labels

In [22]:
df['attitude_themes'] = df['sentence'].apply(predict_theme_category)

In [23]:
df

,author,sentence,attitude_themes
0,Reviewer eS3u,"This work proposes LSTNet, a self-supervised m...",[MET]
1,Reviewer eS3u,"Specifically, LSTNet learns to formulate SO(3)...",[MET]
2,Reviewer eS3u,The proposed self-supervised training pipeline...,"[EXP, MET]"
3,Reviewer eS3u,The self- and cross-reconstruction training st...,[MET]
4,Reviewer eS3u,LSTNet demonstrates state-of-the-art performan...,[DAT]
5,Reviewer eS3u,The performance of aligned shape pairs under t...,[MET]
6,Reviewer eS3u,The reason why other methods are much better t...,[MET]
7,Reviewer eS3u,Lack of limitations.,[MET]
8,Reviewer jP4i,1) This paper proposes a self-supervised metho...,[MET]
9,Reviewer jP4i,2）The main idea is to decouple a point cloud f...,[MET]


In [24]:
def create_clusters(row):
    # Extract root and themes
    root = row["attitude_root"]
    themes = row["attitude_theme"]
    
    # Combine root with each theme
    clusters = [f"{root}({theme})" for theme in themes]
    return clusters

In [25]:
# Apply the function to create clusters
df_cluster = df
df_cluster["clusters"] = df_cluster.apply(create_clusters, axis=1)
# Explode clusters to create one row per cluster
df_cluster = df_cluster.explode("clusters", ignore_index=True)
print(df_cluster)

KeyError: 'attitude_root'